## Curso de Inteligencia Artificial

### Modelo de Markov

Una cadena de Markov (modelo) describe un proceso estocástico donde la probabilidad supuesta de estados futuros depende solo del estado actual del proceso y no de ninguno de los estados que lo precedieron.

### ¿Qué es la propiedad de Markov?

    "... un proceso aleatorio donde el futuro es independiente del pasado dado el presente".

Supongamos un juego de lanzamiento de una moneda simplificado con una moneda justa. Nos gustaría predecir la probabilidad de que salga la cara después de $10$ tiros. Bajo el supuesto de dependencia condicional (la moneda tiene memoria de estados pasados  y el estado futuro depende de la secuencia de estados pasados) debemos registrar la secuencia específica que conduce al undécimo lanzamiento y las probabilidades conjuntas de esos lanzamientos. Así que imagina que después de $10$ lanzamientos tenemos una secuencia aleatoria de caras y sellos. 

La probabilidad conjunta de esa secuencia es $0.5^{10} = 0.0009765625$. Bajo dependencia condicional, la probabilidad de que salga cara en el próximo lanzamiento es $0.0009765625 * 0.5 = 0.00048828125$ .

¿Es esa la probabilidad real de que salga cara en el undécimo lanzamiento?

Sabemos que el evento de lanzar la moneda no depende del resultado del lanzamiento anterior. La moneda no tiene memoria. El proceso de lanzamientos sucesivos no codifica los resultados anteriores. Cada lanzamiento es un evento único con la misma probabilidad de cara o sello, también conocido como condicionalmente independiente de estados pasados. Esta es la propiedad de Markov.



En el ejemplo siguiente, supongamos que deseamos modelar la probabilidad futura de que alguien se encuentra en uno de los tres estados dados del dia y su estado actual de salud. Para hacer esto, necesitamos especificar el espacio de estados, las probabilidades iniciales y las probabilidades de transición.

Estableceremos las probabilidades iniciales en $35\%$, $35\%$ y $30\%$ respectivamente.

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline


estados = ['soleado', 'nublado', 'lloviendo']
pi = [0.35, 0.35, 0.3]
espacio_estados = pd.Series(pi, index=estados, name='estados')
print(espacio_estados)
print(espacio_estados.sum())

El siguiente paso es definir las probabilidades de transición. Son simplemente las probabilidades de permanecer en el mismo estado o pasar a un estado diferente dado el estado actual.

In [ ]:
q_df = pd.DataFrame(columns=estados, index=estados)
q_df.loc[estados[0]] = [0.4, 0.2, 0.4]
q_df.loc[estados[1]] = [0.45, 0.45, 0.1]
q_df.loc[estados[2]] = [0.45, 0.25, .3]

print(q_df)

q = q_df.values
print('\n', q, q.shape, '\n')
print(q_df.sum(axis=1))

Ahora que tenemos la configuración de probabilidades inicial y de transición, podemos crear un diagrama de Markov usando el paquete Networkx.

En el ejemplo, los posibles estados son los nodos y los bordes son las líneas que conectan los nodos. Las probabilidades de transición son los pesos. Representan la probabilidad de pasar a un estado dado el estado actual.

Algo a tener en cuenta es que networkx se ocupa principalmente de objetos de diccionario. Dicho esto, necesitamos crear un objeto de diccionario que contenga nuestros bordes y sus pesos.

In [ ]:
from pprint import pprint 

def _bordes_markov(Q):
    bordes = {}
    for col in Q.columns:
        for idx in Q.index:
            bordes[(idx,col)] = Q.loc[idx,col]
    return bordes

pesos_bordes = _bordes_markov(q_df)
pprint(pesos_bordes)

Para visualizar un modelo de Markov necesitamos usar `nx.MultiDiGraph()`. Un multidigrafo es simplemente un grafo dirigido que puede tener múltiples arcos de modo que un solo nodo puede ser tanto el origen como el destino.

En el siguiente código, creamos el objeto grafo, agregamos los nodos, bordes y etiquetas, luego dibujamos un grafo de networkx (no muy bueno)  mientras generamos nuestro grafo en un archivo `.dot`.


In [ ]:
G = nx.MultiDiGraph()

G.add_nodes_from(estados)
print(f'Nodos:\n{G.nodes()}\n')

for k, v in pesos_bordes.items():
    origen_temporal, destino_temporal = k[0], k[1]
    G.add_edge(origen_temporal, destino_temporal, weight=v, label=v)
print(f'Bordes:')
pprint(G.edges(data=True))    

pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='dot')
nx.draw_networkx(G, pos)

etiquetas_bordes = {(n1,n2):d['label'] for n1,n2,d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G , pos, edge_labels=etiquetas_bordes)
nx.drawing.nx_pydot.write_dot(G, 'markov.dot')

De los resultos, por ejemplo, si está soleado, podemos ver que hay un $40\%$ de probabilidad de que  siga soleado, un $40\%$ de probabilidad de que el clima cambie a nublado y un $20\%$ de probabilidad de que llueva.

## Modelos Ocultos de Markov

Consideramos una situación en la que no conocemos el verdadero estado de alguien (puede ser una enfermedad), por lo que se le oculta. Una forma de modelar esto es asumir que alguien tiene comportamientos observables que representan el verdadero estado oculto. Veamos un ejemplo.

Primero creamos nuestro espacio de estado, `saludable` o `enfermo`. Asumimos que son equiprobables.

In [ ]:
estados_ocultos = ['saludable', 'enfermo']
pi = [0.5, 0.5]
espacio_estados = pd.Series(pi, index=estados_ocultos, name='estados')
print(espacio_estados)
print('\n', espacio_estados.sum())


A continuación, creamos nuestra matriz de transición para los estados ocultos.

In [ ]:
a_df = pd.DataFrame(columns=estados_ocultos, index=estados_ocultos)
a_df.loc[estados_ocultos[0]] = [0.7, 0.3]
a_df.loc[estados_ocultos[1]] = [0.4, 0.6]

print(a_df)

a = a_df.values
print('\n', a, a.shape, '\n')
print(a_df.sum(axis=1))

Ahora creamos la matriz de probabilidad de emisión u observación. Esta matriz es de tamaño $M \times O$ donde $M$ es el número de estados ocultos y $O$ es el número de estados observables posibles.

La matriz de emisión nos dice la probabilidad de que alguien esté en uno de los estados ocultos, dado el estado observable actual.

Mantengamos los mismos estados observables del ejemplo anterior.

In [ ]:
estados_observables = estados

b_df = pd.DataFrame(columns=estados_observables, index=estados_ocultos)
b_df.loc[estados_ocultos[0]] = [0.2, 0.6, 0.2]
b_df.loc[estados_ocultos[1]] = [0.4, 0.1, 0.5]

print(b_df)

b = b_df.values
print('\n', b, b.shape, '\n')
print(b_df.sum(axis=1))

Ahora creamos los bordes del grafo y el objeto grafo.

In [ ]:
pesos_bordes_ocultos = _bordes_markov(a_df)
pprint(pesos_bordes_ocultos)

pesos_bordes_emitido = _bordes_markov(b_df)
pprint(pesos_bordes_emitido)

In [ ]:
G = nx.MultiDiGraph()

G.add_nodes_from(estados_ocultos)
print(f'Nodos:\n{G.nodes()}\n')

for k, v in pesos_bordes_ocultos.items():
    origen_temporal, destino_temporal = k[0], k[1]
    G.add_edge(origen_temporal, destino_temporal, weight=v, label=v)

for k, v in pesos_bordes_emitido.items():
    origen_temporal, destino_temporal = k[0], k[1]
    G.add_edge(origen_temporal, destino_temporal, weight=v, label=v)
    
print(f'Bordes:')
pprint(G.edges(data=True))    

pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='neato')
nx.draw_networkx(G, pos)

etiquetas_bordes_emitidos = {(n1,n2):d['label'] for n1,n2,d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G , pos, edge_labels=etiquetas_bordes_emitidos)
nx.drawing.nx_pydot.write_dot(G, 'markov_oculto.dot')


El grafo de Markov oculto es un poco más complejo, pero los principios son los mismos. Por ejemplo, cabría esperar que si alguien es saludable hay una alta probabilidad de que esté nublado ($60\%$) y una probabilidad muy baja de que el alguien esté enfermo ($10\%$).

Ahora, ¿qué pasaría si necesitara discernir la salud de alguien a lo largo del tiempo dada una secuencia de observaciones?

In [ ]:
obs_map = {'lluvioso':0, 'soleado':1, 'nublado':2}
obs = np.array([1,1,2,1,0,1,2,1,0,2,2,0,1,0,1])

inv_obs_map = dict((v,k) for k, v in obs_map.items())
obs_seq = [inv_obs_map[v] for v in list(obs)]

print( pd.DataFrame(np.column_stack([obs, obs_seq]), 
                columns=['Obs_code', 'Obs_seq']) )

### Algoritmo de Viterbi

Usando el algoritmo de Viterbi podemos identificar la secuencia más probable de estados ocultos dada la secuencia de observaciones.

En el nivel alto, el algoritmo de Viterbi incrementa cada paso de tiempo, encontrando la probabilidad máxima de cualquier camino que llegue al estado `i` en el tiempo `t`, que también tiene las observaciones correctas para la secuencia hasta el tiempo `t`.

El algoritmo también realiza un seguimiento del estado con mayor probabilidad en cada etapa. Al final de la secuencia, el algoritmo iterará hacia atrás seleccionando el estado que  'ganó' en cada paso de tiempo y así creará la ruta más probable, o la secuencia probable de estados ocultos que llevaron a la secuencia de observaciones.

In [ ]:
def viterbi(pi, a, b, obs):
    
    nEstados = np.shape(b)[0]
    T = np.shape(obs)[0]
    ruta = np.zeros(T, dtype=int)
    # delta --> mayor probabilidad de cualquier ruta que llegue, al estado i
    delta = np.zeros((nEstados, T))
    # phi --> argmax por paso de tiempo para cada estado
    phi = np.zeros((nEstados, T))
    
    # Inicio delta y  phi 
    delta[:, 0] = pi * b[:, obs[0]]
    phi[:, 0] = 0

    print('\nInicio del Forward\n')    
    # Extension del algoritmo Forward
    for t in range(1, T):
        for s in range(nEstados):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obs[t]] 
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
            print('s={s} y t={t}: phi[{s}, {t}] = {phi}'.format(s=s, t=t, phi=phi[s, t]))
    
    # Encontrando la ruta optima
    print('-'*50)
    print('Backtrace\n')
    ruta[T-1] = np.argmax(delta[:, T-1])
    for t in range(T-2, -1, -1):
        ruta[t] = phi[ruta[t+1], [t+1]]
        print('ruta[{}] = {}'.format(t, ruta[t]))
        
    return ruta, delta, phi

ruta, delta, phi = viterbi(pi, a, b, obs)
print('\nMejor ruta de estado: \n', ruta)
print('delta:\n', delta)
print('phi:\n', phi)

Echemos un vistazo al resultado:

In [ ]:
mapa_estado = {0:'saludable', 1:'enfermo'}
ruta_estado = [mapa_estado[v] for v in ruta]

(pd.DataFrame()
 .assign(Observaciones=obs_seq)
 .assign(Mejor_ruta=ruta_estado))